In [1]:
import mtt
from os.path import join, realpath
from os import getcwd
sbfile = join(realpath(getcwd()),'fig-3-5a-cascade-irreversible.sb')
wiring = mtt.Wiring.fromFile2(sbfile)
wiring

In [2]:
model = mtt.MTT(wiring)
model.draw(3)

# Digitized current config

In [3]:
for block in wiring.blocks:
    print(block.getDigitizedParameterString(model))

# Generate SRAM Program

In [4]:
from mtt import addSRAMRule

block_indices = [15, 4, 10]
# block_indices = [2, 17, 8]
block_indices = [12, 9, 11]
block_indices = [5, 0, 16]

program = model.compileSRAM()
rules = program.getRoutingRulesBinaryMatrix()

# ADC connections
# group, block, variable, wire
# variables:
Ctot_var  = 11
Afree_var = 12
g = [int(blk/5) for blk in block_indices]
b = [blk%5 for blk in block_indices]
rules = addSRAMRule(rules,g[1],b[1],Afree_var,88) # "A"
rules = addSRAMRule(rules,g[2],b[2],Afree_var,89) # "B"
rules = addSRAMRule(rules,g[2],b[2],Ctot_var,90)  # "C"

# wire to ADC
for i in range(76,99+1):
    rules = addSRAMRule(rules,0,0,31,i)

In [5]:
import pprint
pp = pprint.PrettyPrinter(indent=2, depth=1, width=10)
pp.pprint(program.dump())

# Routing Configuration

One row per wire.

In [6]:
from numpy import array
binrules = array([bin(rule[0])[2:].zfill(10) for rule in rules])
print(binrules.reshape((-1,1)))

# SRAM Layout

Rows are binary routing config rules, cols are wires.

In [7]:
from scipy.io import savemat
sram_matrix = program.convertToSRAMMatrix(rules)
savemat('sram.mat', {'in_all': sram_matrix})

# Shift Registers

In [8]:
sr = model.compileShiftReg()
prog_vect = sr.getProgrammingVector()
for k in range(4):
    prog_vect[280:287,k] = model.digitize(10) # Ione
# import numpy
# numpy.set_printoptions(threshold=numpy.nan)
# print(prog_vect)
savemat('sr.mat', {'prog_vect': prog_vect})

# Simulink Export

In [9]:
model.exportToSimulink(join(realpath(getcwd()),'simulink'), 'cascade', 3)

# Simulation

In [12]:
# hardware simulation
model.reset()
for k in range(3):
    print('block {}: rate_fw = {}'.format(k,model.blocks[k].rate_fw))
    print('block {}: rate = {}'.format(k,model.blocks[k].rate))
r = model.simulate(0,10,100)
# print(r)
model.plot()

# SBML Simulation

In [1]:
# SBML comparison
import tellurium as te
with open('/home/poltergeist/devel/src/mtt/test-cases/fig-3-5a-cascade-reversible.sb') as f:
    rr_model = te.loada(f.read())
print(rr_model.getReactionRates())
rr_model.simulate(0,10,1000)
rr_model.plot()